In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.feature_selection as fs
import pickle
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from sklearn.svm import SVR, NuSVR
from sklearn.pipeline import make_pipeline

In [ ]:
phil = pd.read_csv("../datasets/Philly/DO_QAQC.csv")
phil['DateTime_EST'] = pd.to_datetime(phil['DateTime_EST'])
phil = phil.rename(columns={'DateTime_EST': 'time'}, inplace=False)
phil = phil[phil['Site'] == 'U_A_0']
print(len(phil))

philweather = pd.read_csv("../datasets/Philly/phillyweather.csv")
philweather['time'] = pd.to_datetime(philweather['time'])
print(len(philweather))

In [ ]:
# create new column in philweather called roundedNearestHour and input weather data
# time,temperature_2m (°C),relativehumidity_2m (%),precipitation (mm),surface_pressure (hPa),windspeed_10m (km/h),direct_radiation (W/m²),diffuse_radiation (W/m²)
phil['roundedNearestHour'] = phil['time'].dt.round('H')
phil['temperature_2m (°C)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['temperature_2m (°C)'])
phil['relativehumidity_2m (%)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['relativehumidity_2m (%)'])
phil['precipitation (mm)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['precipitation (mm)'])
phil['surface_pressure (hPa)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['surface_pressure (hPa)'])
phil['windspeed_10m (km/h)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['windspeed_10m (km/h)'])
phil['direct_radiation (W/m²)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['direct_radiation (W/m²)'])
phil['diffuse_radiation (W/m²)'] = phil['roundedNearestHour'].map(philweather.set_index('time')['diffuse_radiation (W/m²)'])


phil.head(10)

In [ ]:
phil.rename(columns={
    'Temp_deg_C': 'temperature',
    'temperature_2m (°C)': 'airtemp',
    'Depth_m': 'Depth',
    'diffuse_radiation (W/m²)': 'light'
}, inplace=True)
phil.columns

In [ ]:
phil['Depth'] = 1.2192
phil['temperature'] = pd.to_numeric(phil['temperature'])
phil['temperature^2'] = phil['temperature'] * phil['temperature']
phil['airtemp^2'] = phil['airtemp'] * phil['airtemp']
phil['temp*airtemp'] = phil['temperature'] * phil['airtemp']
phil['depth*temp'] = phil['Depth'] * phil['temperature']

# Feature Selection

In [ ]:
nf =  pd.read_csv("../datasets/Philly/new_features.csv")
nf['time'] = phil['time']

nf.columns

In [ ]:
'''
nf.rename(columns={
    'Temp_deg_C^2': 'temperature^2',
    'temperature_2m^2': 'airtemp^2',
    'Temp_2m_interaction': 'temp*airtemp',
    'Depth_Temp_interaction': 'depth*temp',
    'Depth_m': 'Depth',
    'windspeed_10m (km/h)': 'windspeed_10m (km/h)',
    'diffuse_radiation': 'light'
}, inplace=True)
'''

X_train, X_test, y_train, y_test = train_test_split(
    phil[['temperature^2', 'airtemp^2', 'temp*airtemp', 'depth*temp', 'Depth', 'windspeed_10m (km/h)', 'light']],
    phil['DO_mg_L'],
    test_size=0.2
)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import math

def stats(y_pred_all, y_test_all):
    # Calculate the R2 score
    r2 = r2_score(y_test_all, y_pred_all)

    print(f"R2 Score: {r2:.4f}")
    # Calculate MAE
    mae = mean_absolute_error(y_test_all, y_pred_all)

    # Calculate RMSE
    mse = mean_squared_error(y_test_all, y_pred_all, squared=False)

    print("Mean Absolute Error (MAE):", mae)
    print("Root Mean Squared Error (RMSE):", math.sqrt(mse))

In [ ]:
print("------------ Random Forest Results ------------")
rf = RandomForestRegressor(n_estimators=24, max_depth=40, random_state=0)
rf.fit(X_train, y_train)
print(rf.feature_importances_)
y_pred = rf.predict(X_test)
stats(y_pred, y_test)

print("------------ Decision Tree Results ------------")
from sklearn.tree import DecisionTreeRegressor
d_tree = DecisionTreeRegressor(max_depth=24)
d_tree.fit(X_train, y_train)
y_pred = d_tree.predict(X_test)
stats(y_pred, y_test)

print("------------ XG Boost Results ------------")
from sklearn.ensemble import GradientBoostingRegressor
xg_boost = GradientBoostingRegressor(learning_rate=0.1, loss='huber', max_depth=6, criterion='squared_error')
xg_boost.fit(X_train, y_train)
y_pred = xg_boost.predict(X_test)
stats(y_pred, y_test)

In [ ]:
print("------------ Poly Regression Results ------------")
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=12, include_bias=True)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)
# Initialize and fit the linear regression model
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train)
# Predict the target variable for training and test sets
y_pred = poly_reg.predict(X_test_poly)
stats(y_pred, y_test)

In [ ]:
print("------------ Exponential SVR Results ------------")
clf_rbf_nusvm = make_pipeline(preprocessing.SplineTransformer(), NuSVR(kernel='rbf', shrinking=True, C=1.5))
clf_rbf_nusvm.fit(X_train, y_train)
y_pred = clf_rbf_nusvm.predict(X_test)
stats(y_pred, y_test)

print("------------ Exponential SVR Results ------------")
clf_rbf_svm = make_pipeline(preprocessing.SplineTransformer(), SVR(kernel='rbf', shrinking=True, C=1.5))
clf_rbf_svm.fit(X_train[:len(X_train)//2], y_train[:len(X_train)//2])
clf_rbf_svm.fit(X_train[len(X_train)//2:], y_train[len(X_train)//2:])
y_pred = clf_rbf_svm.predict(X_test)
stats(y_pred, y_test)

print("------------ Polynomial SVR Results ------------")
clf_poly_nusvm = make_pipeline(preprocessing.SplineTransformer(), NuSVR(kernel='poly', shrinking=False, C=2.5))
clf_poly_nusvm.fit(X_train, y_train)
y_pred = clf_poly_nusvm.predict(X_test)
stats(y_pred, y_test)

In [ ]:
print("------------ Bagging Results ------------")
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
base_estimator = DecisionTreeRegressor()
bag_pipe = make_pipeline(preprocessing.SplineTransformer(), BaggingRegressor(base_estimator=base_estimator, n_estimators=10, n_jobs=5))
bag_pipe.fit(X_train, y_train)
y_pred = bag_pipe.predict(X_test)
stats(y_pred, y_test)

print("------------ Extra Trees Results ------------")
from sklearn.ensemble import ExtraTreesRegressor
extra_pipe = make_pipeline(preprocessing.SplineTransformer(), ExtraTreesRegressor(n_estimators=10, n_jobs=5))
extra_pipe.fit(X_train, y_train)
y_pred = extra_pipe.predict(X_test)
stats(y_pred, y_test)

print("------------ Ada Boost Results ------------")
from sklearn.ensemble import AdaBoostRegressor
ada_pipe = make_pipeline(preprocessing.SplineTransformer(), AdaBoostRegressor(base_estimator=base_estimator, n_estimators=10))
ada_pipe.fit(X_train, y_train)
y_pred = ada_pipe.predict(X_test)
stats(y_pred, y_test)

'''
print("------------ Voting Results ------------")
from sklearn.ensemble import VotingRegressor
voting_pipe = make_pipeline(preprocessing.SplineTransformer(), VotingRegressor(estimators=[
    ('svr', NuSVR(kernel='poly', shrinking=False, C=2.5)),
    ('rf', RandomForestRegressor(n_estimators=10,random_state=42, n_jobs=8)),
    ('bag', BaggingRegressor(n_jobs=5)),
    ('bst', GradientBoostingRegressor(learning_rate=0.1, loss='huber', max_depth=6, criterion='squared_error')),
    ('n1', MLPRegressor(hidden_layer_sizes=(2,3), activation='relu')),  
    ('n2', MLPRegressor(hidden_layer_sizes=(3,2), activation='tanh'))
]))
voting_pipe.fit(X_train, y_train)
y_pred = voting_pipe.predict(X_test)
stats(y_pred, y_test)


print("------------ Stacking Results ------------")
from sklearn.ensemble import StackingRegressor
model = StackingRegressor(estimators=[
    ('svr', NuSVR(kernel='poly', shrinking=False, C=2.5)), 
    ('rf', RandomForestRegressor(n_estimators=10,random_state=42, n_jobs=8)),
    ('bag', BaggingRegressor(n_jobs=5)),
    ('bst', GradientBoostingRegressor(learning_rate=0.1, loss='huber', max_depth=6, criterion='squared_error')),
    ('n1', MLPRegressor(hidden_layer_sizes=(2,3), activation='relu')),
    ('n2', MLPRegressor(hidden_layer_sizes=(3,2), activation='tanh'))
])
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
stats(y_pred, y_test)
'''


# Connect to DB

In [ ]:
import pymysql

# Connect to the database
connection = pymysql.connect(
    host='localhost', 
    user='root', 
    password='N@wid2003', 
    db='dma_iot_morefish_spark_farms_v3'
)

# Create a cursor object
cursor = connection.cursor()

# Get the latest data from the database and store it in a pandas dataframe
query = "SELECT dvd_ph, dvd_temp, dvd_updated_at, dvd_do, dvd_dev_id FROM device_devicedata WHERE dvd_ph > 0 AND dvd_temp > 0 AND dvd_dev_id = 2 ORDER BY dvd_updated_at ASC"
df = pd.read_sql(query, connection)

# Close the connection
connection.close()
len(df)

In [9]:
excelData = pd.read_excel("../../../../../../Downloads/device_data_july13.xlsx")
excelData.to_csv("../datasets/Philly/device_data_july13.csv")


In [ ]:
df.rename(columns={'dvd_ph': 'ph', 'dvd_temp': 'temperature', 'dvd_updated_at': 'datetime', 'dvd_do': 'do_linreg'}, inplace=True)
df['rounded_datetime'] = pd.to_datetime(df['datetime']).dt.round('H')
# spark_weather = pd.read_csv("../../../../../../Downloads/sparkfarmsweather.csv")
spark_weather = pd.read_csv("../datasets/sparkfarmsweather.csv")
spark_weather['time'] = pd.to_datetime(spark_weather['time'])
spark_weather.rename(columns={'time': 'datetime'}, inplace=True)
# Merge the weather data with the main DataFrame based on 'rounded_datetime' column
df = df.merge(spark_weather, left_on='rounded_datetime', right_on='datetime', how='left')
df = df.drop(columns=['datetime_y'])
df.rename(columns={'datetime_x': 'datetime', 'diffuse_radiation (W/m²)': 'light', 'relativehumidity_2m (%)': 'humid', 'temperature_2m (°C)': 'airtemp' }, inplace=True)
# drop all NaN values inside diffuse radiation
df = df.dropna(subset=['light'])
df.columns

In [ ]:
# Add new features: depth, temperature^2, airtemp^2, temp*airtemp, depth*temp

df['Depth'] = 1.2192
df['temperature'] = pd.to_numeric(df['temperature'])
df['temperature^2'] = df['temperature'] * df['temperature']
df['airtemp^2'] = df['airtemp'] * df['airtemp']
df['temp*airtemp'] = df['temperature'] * df['airtemp']
df['depth*temp'] = df['Depth'] * df['temperature']

In [ ]:
sf_test = df[['temperature^2', 'airtemp^2','temp*airtemp', 'depth*temp', 'Depth', 'windspeed_10m (km/h)', 'light']]
y_pred = poly_reg.predict(poly_features.transform(sf_test))
df['do_poly'] = y_pred
df[['do_linreg', 'do_poly']].sample(10)

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

myDates = [datetime(2012,1,i+3) for i in range(10)]
myValues = [5,6,4,3,7,8,1,2,5,4]
fig, ax = plt.subplots()
bet = df.loc[df['datetime'].between('2023-06-10', '2023-06-13')]
ax.plot(bet['datetime'], bet['do_poly'].astype(float), 'bo')

myFmt = DateFormatter("%m %d %H:%M")
ax.xaxis.set_major_formatter(myFmt)

## Rotate date labels automatically
fig.autofmt_xdate()
plt.show()